# 认证

即使在“普通”的 OpenSSH 客户端中，向远程服务器认证也涉及多个密钥和配置的可能来源。Fabric 不仅支持其中的大多数功能，而且还有更多自己的功能。本文档概述了设置认证密钥的可用方法。

```{note}
由于 Fabric 本身尽量不重新发明 Paramiko 的功能，因此大部分时间配置认证值归结于“如何为 {meth}`paramiko.client.~SSHClient.connect` 设置关键字参数的值”，这反过来意味着要在 `connect_kwargs` [配置子树](https://docs.fabfile.org/en/latest/concepts/configuration.html)内或 {class}`~fabric.connection.Connection` 的 `connect_kwargs` 关键字参数中设置值。
```

## 私钥文件

存储在磁盘上的私钥文件可能是 SSH 最普遍的认证机制。Fabric 提供了多种配置方法，用于指定使用哪些路径，这些路径最终合并成一个列表，传递给 `SSHClient.connect(key_filename=[...])` 函数，顺序如下：

- 如果在 `Connection` 的 `connect_kwargs` 参数中存在 `key_filename` 键，它们会在列表中排在首位。（这基本上是非 CLI 用户的“运行时”选项。）
- [`config` 设置](https://docs.fabfile.org/en/latest/concepts/configuration.html)中的 `connect_kwargs.key_filename` 可以通过多种方式设置（如配置文档所述），包括通过 `--identity` CLI标志（该标志设置了配置的覆盖级别；因此，当使用此标志时，来自其他配置源的密钥文件名值将被覆盖）。这个值在整体列表中排在第二位。
- 使用带有 `IdentityFile` 指令的 [ssh_config 文件](https://docs.fabfile.org/en/latest/concepts/configuration.html#ssh-config)，可以让你与其他 SSH 客户端共享配置；这样的值排在最后。

## 加密密码短语

如果你的私钥文件通过密码短语(Encryption passphrases)来保护，可以通过几种方式提供这个密码短语：

- `connect_kwargs.passphrase` 配置选项是最直接的方式来自动提供密码短语。

```{note}
对于这类材料使用实际的磁盘配置文件并不总是明智的选择，但请记住，[配置系统](https://docs.fabfile.org/en/latest/concepts/configuration.html)能够从其他来源加载数据，比如你的shell环境或甚至是任意的远程数据库。
```

如果你更喜欢在运行时手动输入密码短语，你可以使用命令行选项 `--prompt-for-passphrase`，这将使得Fabric在过程开始时交互式地提示用户，并将输入的值存储在 `connect_kwargs.passphrase`（在“覆盖”级别）。

## **私钥对象**
实例化你自己的 {class}`~paramiko.pkey.PKey` 对象（详情请查看其子类的API文档），并将其放入 `connect_kwargs.pkey` 中。就这样！如果你加载的密钥材料（这些类可以从文件路径或字符串中加载）是加密的，你将负责处理任何密码短语。

## **SSH代理**
默认情况下（类似于OpenSSH的行为），Paramiko将尝试连接到一个正在运行的SSH代理（Unix风格，例如一个活动的SSH_AUTH_SOCK，或者如果是在Windows上则可能是Pageant）。可以通过将 `connect_kwargs.allow_agent` 设置为 `False` 来禁用此功能。

## **密码**
密码认证相对直接：

- 你可以直接通过 `connect_kwargs.password` 来配置它。
- 如果你想在会话开始时被提示输入密码，指定 `--prompt-for-login-password`。

## **GSSAPI**
Fabric并没有在Paramiko现有的连接时间参数之上提供任何额外的GSSAPI支持（例如在 `SSHClient.connect` 中看到的 `gss_kex/gss_auth/gss_host/` 等），以及实现该功能的模块本身（如 `paramiko.ssh_gss`）。因此，像往常一样，你应该着手修改 `connect_kwargs` 配置树。